# IBM Ponder This - August 2024

## Problem Statement

This month’s challenge was inspired by the New York Times game, Connections. In our version of this popular game, players are presented with 16 items and asked to split them into 4 sets of 4 items each, where the elements of each set have something in common. For example, consider the following elements:

<div class="center">
<img src="images/August2024-1.png" alt="0114.png", width="400" height="250">
</div>

The intended way to split them into sets can be demonstrated using the following coloring:

<div class="center">
<img src="images/August2024-2.png" alt="0114.png", width="400" height="250">
</div>

where the sets are mathematical constants, non-abelian groups, number-theoretic functions, and asymptotic notations.

Suppose a player has no idea what the elements mean and simply categorizes the 16 elements into 4 sets at random. If the player guessed a set correctly, it is marked and removed from the game, and the player keeps guessing at random (and may even repeat previous failed guesses) until all the correct sets were guessed. We call each time the player chooses a random partition and receives a feedback on the choice "a step".

Note: unlike the original game, in this game the player chooses a full partition of all the remaining elements into sets, not just choosing elements for one set. So in the beginning the player is making 4 guesses at once.

Your goal: Find the expected number of steps in such a blind game, given a player's totally random behavior. You can round to the nearest integer.

A bonus "*" will be given for solving the same problem, but for a game consising of 50 items, categorized into 10 sets of 5 elements each.

## Solution

Let $N_g$ denote the number of groups and $N_e$ denote the number of elements per group. Without loss of generality, we can assume that the $N_g$ sets are numbered from 1 to $N_g$. Also, we can number each element from 1 to $N_gN_e$ and say that element $i$ belongs to the set numbered $\left \lfloor \frac{i - 1}{N_e} \right \rfloor  + 1$. When there are $i$ remaining sets, we can assume that the player has $i$ baskets in front of him. He first selects $N_e$ numbers and put them in the first basket. Then, he selects $N_e$ numbers and put them in the second basket. He repeats the process until there are no remaining numbers and all the baskets are filled with $N_e$ numbers. When all the $N_e$ numbers belong to the same valid set, this set is removed from the game.

Let $A$ be an $(N_g + 1) \times (N_g + 1)$ matrix where $A_{i, j}$ indicates the number of ways in which we can go $i$ remaining sets to $j$ remaining sets for $0 \leq i, j \leq N_g$. 

As it is not possible to end up with more remaining sets than before playing, all the entries above the main diagonal are filled with 0. Also, it is not possible to end up (or start) with exactly one remaining set. This is because if there are 2 remaining sets and we fill one, we necessarily fill the second one too. Therefore, the second row and column are filled with 0 too.

Then we can consider two separate case. The first case is when we end up with less remaining sets that we start with. If we go from $i$ remaining sets to $j$ remaining sets, it means we have filled $k = i - j$ sets and $A_{i, j}$ is given by

\begin{equation}
    A_{i, j} = \binom{i}{k}^2 k! N_e! A_{j, j}.
\end{equation}

The first $\binom{i}{k}$ term is the number of ways to select $k$ valid sets among the $i$ valid sets (i.e., set of numbers belonging to the same set). The second $\binom{i}{k}$ term is the number of ways to select the empty baskets in which we put the numbers. The $k!$ term represent the different orderings of the $k$ valid set of numbers within the selected baskets. The $N_e$ term represents the number of orderings of $N_e$ elements in the basket. Finally, the last term represents the number of ways to arange the remaining elements (those that are not in valid sets). This value is the number of ways to arange the $jN_e$ remaining elements from which we subtract all the arrangements that would lead to a different number of valid sets (following the inclusion exclusion principle). This value is therefore equal to the number of ways in which we can go from $j$ sets to $j$ sets (i.e., we start and end with $j$ sets). 

The second case is the one we just mentioned, when we start and end with the same number of remaining sets. This is the diagonal part of $A$. This is simply equal to the number of arrangements of all the remaining elements minus the arrangements with any number of valid sets. This is given by

\begin{equation}
    A_{i, i} = (iN_e)! - \sum_{j < i} A{i}{j}.
\end{equation}

This matrix gives the number of ways to move from $i$ remaining sets to $j$ remaining sets. However, we are interested in the probability of moving from $i$ remaining sets to $j$ remaining sets. We can convert the matrix $A$ to a probability matrix $P$ as

\begin{equation}
    P_{i, j} = \frac{A_{i, j}}{(iN_e)!}.
\end{equation}

Our matrix is now a transition matrix. We just need to rotate it by 180 degrees to have it in the standard format. From here, we can get the expected number of steps from markov chain theory. We first decompose our matrix $P$ as

\begin{equation}
    P = \begin{bmatrix}
    Q & R \\
    0 & 1
    \end{bmatrix}
\end{equation}

where $Q$ is a $(n-1) \times (n-1)$ matrix representing transitions among the transient states, $R$ is a $(n-1) \times 1$ matrix representing the probabilities of transitioning from each transient state to the absorbing state, $0$ is a $1 \times (n-1)$ matrix of zeros, indicating no transitions from the absorbing state back to any transient states, $1$ indicates the self-loop of the absorbing state, staying in itself with probability 1.

Then we compute the fundamental matrix 

\begin{equation}
    N = (I - Q)^{-1}
\end{equation}

where $I$ is the identify matrix of size $n - 1$.

The expected number of steps from each transient state to the absorbing state is now computed as

\begin{equation}
    t = N \times \bold{1}
\end{equation}

where $\bold{1}$ is a vector of ones of dimention $n - 1$. This effectively sums each row of $N$ and each sum indicates the expected number of steps from each transient state to an absorbing state. 

Therefore, the expected number of steps we are interested in is given by $t_0$.


In [1]:
import numpy as np
import math

def expected_steps_to_absorb(transition_matrix):
    # Convert the input list to a NumPy array if it's not already one
    P = np.array(transition_matrix)

    # Number of states (assuming the matrix is square)
    n = P.shape[0]

    # Extract matrix Q from P (excluding the last row and column)
    Q = P[:-1, :-1]

    # Compute the identity matrix of size (n-1)
    I = np.eye(n - 1)

    # Compute the fundamental matrix N = (I - Q)^(-1)
    N = np.linalg.inv(I - Q)

    # Compute the expected number of steps for each transient state
    t = N.sum(axis=1)

    # Return the expected number of steps from the initial state
    return t[0]  # Assuming the initial state is the first state


n_groups = 10
n_elements = 5

def generate_transition_matrix(n_groups, n_elements):

    # Compute number of ways to go from i remaining sets to j remaining sets
    dp = [[0] * (n_groups + 1) for _ in range(n_groups + 1)]
    dp[0][0] = 1
    for i in range(2, n_groups + 1):
        for j in range(i + 1):
            if i != 0:
                if i == j:
                    dp[i][j] = math.factorial(i * n_elements) - sum(dp[i][:i])
                else:
                    k = i - j
                    dp[i][j] = math.factorial(k) * math.comb(i, k)**2 * math.factorial(n_elements)**k * dp[j][j]

    # Convert to probabilities
    for i in range(2, n_groups + 1):
        for j in range(i + 1):
            dp[i][j] /= math.factorial(i * n_elements)

    # Rotate transition matrix to standard form
    dp = np.array(dp)
    dp = np.flipud(np.fliplr(dp))

    return dp


n_groups = 4
n_elements = 4
transition_matrix = generate_transition_matrix(n_groups, n_elements)
expected_steps = expected_steps_to_absorb(transition_matrix)
print("Expected number of steps to reach the absorbing state:", expected_steps)

Expected number of steps to reach the absorbing state: 205.00613237013636


In [2]:
n_groups = 10
n_elements = 5
transition_matrix = generate_transition_matrix(n_groups, n_elements)
expected_steps = expected_steps_to_absorb(transition_matrix)
print("Expected number of steps to reach the absorbing state:", expected_steps)

Expected number of steps to reach the absorbing state: 60694.39326815214
